# 5. PROGRAMMING WITH RDDS

## RDD란?
RDD는 Resilient Distributed Dataset의 줄임말로, objects sets의 immutable distributed collection이다.
각 RDD는 여러개의 파티션으로 나누어지고, 각 파티션은 클러스터의 여러 노드에 의해 분배된다.

## 5.1 Create RDD

방법 1: parallelize 사용

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("chapter5").getOrCreate()

df = spark.sparkContext.parallelize(
    [(1, 2, 3, "a b c"), (4, 5, 6, "d e f"), (7, 8, 9, "g h i")]
).toDF(["col1", "col2", "col3", "col4"])

df.show()

25/08/10 17:42:40 WARN Utils: Your hostname, jamess-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.29.112.67 instead (on interface en0)
25/08/10 17:42:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/08/10 17:42:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----+----+----+-----+
|col1|col2|col3| col4|
+----+----+----+-----+
|   1|   2|   3|a b c|
|   4|   5|   6|d e f|
|   7|   8|   9|g h i|
+----+----+----+-----+



In [ ]:
df.collect()

2. createDataFrame() 사용

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("chapter5").getOrCreate()

Employee = spark.createDataFrame(
    [
        (1, "James", "Smith", "1991-04-01", "M", 3000),
        (2, "Michael", "Rose", "2000-05-19", "M", 4000),
        (3, "Robert", "Williams", "1978-09-05", "M", 4000),
    ],
    ["id", "firstname", "lastname", "dob", "gender", "salary"],
)

Employee.show()

Employee.printSchema()

3. read / load를 이용해 파일을 읽어오기

In [ ]:
# 로컬에 있는 데이터 파일을 읽어오기
df = (
    spark.read.format("com.databricks.spark.csv")
    .options(header="true", inferSchema="true")
    .load("./LearningApacheSpark/chapter5/data.csv")
)

df.show()
df.printSchema()

In [ ]:
# 데이터베이스 있는 파일 읽어오기

# jdbc(java database connectivity)를 이용
url = f"jdbc:mysql://localhost:3306/mydb"

df = (
    spark.read.format("jdbc")
    .options(
        url=url,
        driver="com.mysql.cj.jdbc.Driver",
        dbtable="users",
        user="root",
        password="password",
    )
    .load()
)

df.show(5)
df.printSchema()

In [ ]:
# HDFS 파일 읽어오기

from pyspark.sql import HiveContext
from pyspark.context import SparkContext
from pyspark.conf import SparkConf

sc = SparkContext("local", "example")
hc = HiveContext(sc)
tf1 = sc.textFile("hdfs://localhost:9000/user/chapter5/data.csv")

# intg_cme_w 데이터베이스 사용
hc.sql("use intg_cme_w")

# 해당 데이터베이스에서 쿼리
spf = hc.sql("select * from spf")
spf.show()

## 5.2 Spark Operations

스파크에서 operations은 크게 두가지 종류 (transformations & actions)로 구분된다.

### 5.2.1 Spark Transformations

transformation은 이전의 RDD를 변형해 새로운 RDD를 생성한다(map에 해당). 아래 사진의 함수들이 존재한다.
![](2025-08-10-17-07-42.png)

### 5.2.2 Spark Actions

action은 반대로 RDD를 aggregation하여 결과물을 얻어낸다. (reduce에 해당)
![](2025-08-10-17-09-24.png)
![](2025-08-10-17-09-30.png)

### 5.3.2 Load DataFrame

In [ ]:
# 아래처럼 로컬 파일을 읽어올 수 있다
ds = spark.read.csv(
    path="Advertising.csv",
    # sep=',',
    # encoding='UTF-8',
    # comment=None,
    header=True,
    inferSchema=True,
)


In [ ]:
# json 파일 읽어오기
!wget https://api.luftdaten.info/static/v1/data.json
ds = spark.read.json("data.json")

In [ ]:
# 4 head row
ds.show(4)

In [ ]:
# data types of each column
ds.dtypes

In [ ]:
# columns
ds.columns

### Fill  Null

In [ ]:
my_list = [["male", 1, None], ["female", 2, 3], ["male", 3, 4]]
ds = spark.createDataFrame(my_list, ["gender", "age", "salary"])
ds.show()


In [ ]:
ds.fillna(-99).show()

In [ ]:
# replace values
# caution: Mixed type replacements are not supported
ds.na.replace(["male", "female"], ["M", "F"]).show()

In [ ]:
# rename columns
ds.toDF("a", "b", "c").show()

ds.withColumnRenamed("gender", "a").show()


In [ ]:
# drop columns
ds.drop("gender").show()

In [ ]:
# filter
ds.filter(ds.gender == "male").show()

# or
ds[ds.gender == "male"].show()

# multiple filter
ds[(ds.gender == "male") & (ds.age > 2)].show()

In [ ]:
from pyspark.sql import functions as F

# make new column
ds.withColumn(
    "age_norm",
    ds.age / ds.groupBy().agg(F.sum("age")).collect()[0][0],  # sum of ages
).show()

In [ ]:
# complicated function
ds.withColumn(
    "cond", F.when(ds.gender == "male", 1).otherwise(0)
).show()  # male: 1, female: 0

In [ ]:
# DataFrame 복사 방법들
new_ds = ds.toDF(*["a", "age", "c"])  # 모든 컬럼 선택으로 복사

# right (left, inner, full도 가능) join
ds.join(new_ds, on="age", how="right").orderBy("A", ascending=True).show()


In [ ]:
# concat two columns
ds.withColumn("concat", F.concat(ds.gender, ds.age)).show()

In [ ]:
# groupby
# null인 값은 무시된다.
ds.groupBy("gender").agg({"age": "sum", "salary": "avg"}).show()

In [ ]:
# pivot
ds.groupBy("gender").pivot("age").agg(F.sum("salary")).show()

In [3]:
# window

import pyspark.sql.functions as F
import pandas as pd

d = {"A": ["a", "b", "c", "d"], "B": ["m", "m", "n", "n"], "C": [1, 2, 3, 6]}
df = pd.DataFrame(d)
ds = spark.createDataFrame(df)

ds.show()

from pyspark.sql.window import Window

w = Window.partitionBy("B").orderBy("A")

ds.withColumn("rank", F.rank().over(w)).show()

/Users/james/Study/pyspark/.venv/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/james/Study/pyspark/.venv/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+---+---+
|  A|  B|  C|
+---+---+---+
|  a|  m|  1|
|  b|  m|  2|
|  c|  n|  3|
|  d|  n|  6|
+---+---+---+

+---+---+---+----+
|  A|  B|  C|rank|
+---+---+---+----+
|  a|  m|  1|   1|
|  b|  m|  2|   2|
|  c|  n|  3|   1|
|  d|  n|  6|   2|
+---+---+---+----+

